In [1]:
import tempfile
temp_dir = tempfile.TemporaryDirectory()
import os
os.chdir("../../")
cwd = ! pwd
cwd = cwd[0]
os.chdir(temp_dir.name)
pyre_config = f"{cwd}/.pyre_configuration"
watchman_config = f"{cwd}/.watchmanconfig"
! cp $pyre_config $temp_dir.name
! cp $watchman_config $temp_dir.name
%env PYTHONPATH=$cwd
import warnings
warnings.filterwarnings('ignore')
print(temp_dir.name)

env: PYTHONPATH=/Users/lomesh/open-source/Fixit
/var/folders/dk/x6hpyx3d3555d6d0b6nhszjw0000gn/T/tmp4x7t6a68


In [2]:
from typing import Dict


class C(object):
    attr = "ab" "cd" "ef" "gh"

    def method(self) -> Dict[int, str]:
        filtered_char = []
        for char in self.attr:
            if char is not "a":
                filtered_char.append(char)

        index_to_char = dict([(idx, char) for idx, char in enumerate(filtered_char)])
        return index_to_char

In [3]:
FILE_NAME = "example.py"
from pathlib import Path
fp = open(Path(temp_dir.name) / FILE_NAME, "w")
fp.write(In[2]+"\n")
fp.close()
! pyre start --store-type-check-resolution
! git init
! git add example.py
! pyre query "types(path='example.py')"

/bin/sh: pyre: command not found
Initialized empty Git repository in /private/var/folders/dk/x6hpyx3d3555d6d0b6nhszjw0000gn/T/tmp4x7t6a68/.git/
/bin/sh: pyre: command not found


In [4]:
! python3.8 -m fixit.cli.run_rules

Scanning 1 files
Testing 23 rules

Encountered exception <class 'Exception'> for the following paths:
./example.py
Running `pyre start` may solve the issue.
example.py:4:1
    NoInheritFromObjectRule: Inheriting from object is a no-op.  'class Foo:' is
    just fine =)
example.py:5:12
    UsePlusForStringConcatRule: Implicit string concatenation detected, please
    add '+' to be explicit. E.g. a tuple or a call ("a" "b") with a missing
    comma results in multiple strings being concatenated as one string and
    causes unexpected behaviour.
example.py:10:16
    ComparePrimitivesByEqualRule: Don't use `is` or `is not` to compare
    primitives, as they compare references. Use == or != instead.
example.py:13:25
    RewriteToComprehensionRule: It's unnecessary to use a list comprehension
    inside a call to dict since there are equivalent comprehensions for this
    type
example.py:10:16
    F632: use ==/!= to compare constant literals (str, bytes, int, float, tuple)
example.py:13:80
 

In [5]:
! python3.8 -m fixit.cli.apply_fix

Scanning 1 files
./example.py

Encountered exception <class 'Exception'> for the following paths:
./example.py
Running `pyre start` may solve the issue.
example.py:4:1 [applied fix]
    NoInheritFromObjectRule: Inheriting from object is a no-op.  'class Foo:' is
    just fine =)
Encountered exception <class 'Exception'> for the following paths:
./example.py
example.py
example.py
Running `pyre start` may solve the issue.
example.py:5:12 [applied fix]
    UsePlusForStringConcatRule: Implicit string concatenation detected, please
    add '+' to be explicit. E.g. a tuple or a call ("a" "b") with a missing
    comma results in multiple strings being concatenated as one string and
    causes unexpected behaviour.
Encountered exception <class 'Exception'> for the following paths:
./example.py
example.py
example.py
example.py
example.py
example.py
Running `pyre start` may solve the issue.
example.py:10:16 [applied fix]
    ComparePrimitivesByEqualRule: Don't use `is` or `is not` to compare
   

In [6]:
! git diff

diff --git a/example.py b/example.py
index aed4bb5..3f667f8 100644
--- a/example.py
+++ b/example.py
@@ -1,14 +1,14 @@
 from typing import Dict
 
 
-class C(object):
-    attr = "ab" "cd" "ef" "gh"
+class C:
+    attr = "ab" + "cd" + "ef" + "gh"
 
     def method(self) -> Dict[int, str]:
         filtered_char = []
         for char in self.attr:
-            if char is not "a":
+            if char != "a":
                 filtered_char.append(char)
 
-        index_to_char = dict([(idx, char) for idx, char in enumerate(filtered_char)])
+        index_to_char = {idx: char for idx, char in enumerate(filtered_char)}
         return index_to_char


In [9]:
! python3.8 -m fixit.cli.add_new_rule --path fixit/rules/my_rule.py
! cat fixit/rules/my_rule.py

All done! ✨ 🍰 ✨
1 file left unchanged.
Successfully created my_rule.py rule file at fixit/rules
# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.


import libcst as cst
import libcst.matchers as m

from fixit import CstLintRule, InvalidTestCase as Invalid, ValidTestCase as Valid


"""
This is a model rule file for adding a new rule to fixit module
"""


class Rule(CstLintRule):
    """
    docstring or new_rule description
    """

    MESSAGE = "Enter rule description message"

    VALID = [Valid("'example'")]

    INVALID = [Invalid("'example'")]


In [ ]:
temp_dir.cleanup()